# Strings

## Die Hilfsklasse Alphabet

In Python (und Java) sind Strings als Arrays Unicode Charactern implementiert. 
Das heisst, das Alphabet kann über 1'000'000 unterschiedliche Zeichen unterscheiden. 
Bei vielen Anwendungen nutzen wir aber nur einen kleinen Teil dieses Alphabets. Da auch die Speicherkomplexität vieler Algorithmen von der Grösse des Alphabets abhängt, macht es Sinn, dieses zu beschränken. Dazu definieren wir uns die Klasse Alphabet.

In [ ]:
class Alphabet:
    
    # Ein Objekt wird durch die im Alphabet zur Verfügung stehenden
    # Zeichen definiert (welche in der Liste chars mitgegeben wird)
    def __init__(self, chars):
        
        # In der Symboltabelle alphabet wird zu jedem Zeichen die 
        # Position im Alphabet gespeichert
        self._alphabet=dict()
        
        # Im Array Inverse wird zu jeder gegebenen Position im Alphabet
        # der entsprechende Buchstabe gespeichert
        self._inverse = [None]*len(chars)        
                
        for (i, c) in enumerate(chars):
            self._alphabet[c] = i
            self._inverse[i] = c
    
    # Gibt den zum gegebenen Index gehörenden Buchstaben zurück
    def toChar(self, index):
        return self._inverse[index]
        
    # Gibt den Index (Position im Alphabet) des Buchstabens zurück
    def toIndex(self, char):
        return self._alphabet[char]
    
    # True, falls der Buchstabe char im Alphabet enthalten ist
    def contains(self, char):
        return self._alphabet.contains(char)

    # Anzahl Zeichen im Alphabet
    def radix(self):
        return len(self._inverse)
    
    

Mithilfe der Alphabet Abstraktion können wir uns nun die wichtigsten Alphabete, mit denen wir im folgenden Arbeiten werden, einfach definieren. 

In [ ]:
lowercase = Alphabet("abcdefghijklmnopqrstuvwxyz")
uppercase = Alphabet("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
basicAscii = Alphabet([chr(i) for i in range(0, 128)])
DNA = Alphabet("ACGT")

Für jedes Alphabet muss die Konsistenzbedingung gelten, dass die Methoden ```toChar``` die Inverse Methode von ```toIndex``` ist, im Sinne, dass ```toIndex(toChar(i))``` jeweils wieder den Index ```i``` ergibt.  

In [ ]:
def conversionProperty(alphabet):
    for i in range(0, alphabet.radix()):
        assert(alphabet.toIndex(alphabet.toChar(i)) == i)

conversionProperty(DNA)


## LSD Sort

Eine einfache Sortiermethode für Strings ist das LSD-Sortierverfahren (siehe Slides für detaillierte Erklärungen). 
Wir nutzen hier die oben eingeführte Alphabet Abstraktion. Die Implementation macht die Annahme, dass alle Strings dieselbe Länge haben. 

In [ ]:
def lsdSort(a, alphabet = basicAscii):
    
    # Annahme, alle Strings haben dieselbe Länge
    for x in a:  
        assert(len(x) == len(a[0]))

    numDigits = len(a[0])
        
    N = len(a)
    aux = [None] * N
    
    d = numDigits - 1
    while d >= 0:
        
        count = [0] * (alphabet.radix() + 1)
        
        for i in range(0, N):
            indexOfcharAtPosdInA = alphabet.toIndex(a[i][d])
            count[indexOfcharAtPosdInA + 1] += 1
        
        for r in range(0, alphabet.radix()):
            count[r+1] += count[r]
        
        for i in range(0, N):
            indexOfCharAtPosdInA = alphabet.toIndex(a[i][d])
            countForChar = count[indexOfCharAtPosdInA]
            aux[countForChar] = a[i]
            count[indexOfCharAtPosdInA] += 1
        
        # zurückkopieren
        for i in range(0, N):
            a[i] = aux[i]

        d -= 1
            

Wir testen unsere Implementation mit einem einfachen Testfall:

In [ ]:
xs = ["th", "ar", "tw", "ty", "of", "mi"]
lsdSort(xs)
print(xs)

*Übung:*  Testen Sie weitere Testfälle. Funktioniert die Implementation auch mit allen Grenzfällen? (Leere Liste, bereits sortierte Sequenzen, ...)

## 3-Weg Quicksort für Strings

Im folgenden zeigen wir, wie wir den 3-Weg String Quicksort aus dem Standard Quicksort Algorithmus herleiten können. 
Da es hier nur darum geht das Prinzip zu illustrieren nutzen wir hier eine vereinfachte Version von Quicksort in der Herleitung. Diese Vereinfachte Version verfolgt dasselbe Prinzip wie der richtige Quicksort Algorithmus, ist aber sehr ineffizient und nicht inplace, wie das richtige Quicksort Verfahren. Sie sollten das Verfahren so also nie in der Praxis einsetzten. 

Zuerst zeigen wir hier wie der Quicksort Algorithmus in dieser vereinfachten Form implementiert werden kann.

Die Funktion ```partition``` partitioniert die Elemente in der Liste xs in zwei Teile, die die kleiner als das Pivotelement sind und die die Grösser oder gleich dem Pivotelement sind.

In [ ]:
def partition(a, pivot):
    smaller = []
    largerOrEqual = []
    for x in a:
        if x < pivot:
            smaller.append(x)
        else:
            largerOrEqual.append(x)
    return (smaller, largerOrEqual)

Der eigentliche Algorithmus lässt sich dann in dieser sehr intuitiven Form hinschreiben:

In [ ]:
def quicksortSimplified(a):
    if len(a) <= 1:
        return a
    
    pivot = a[0]
    (smaller, larger) = partition(a[1:], pivot)     
    return quicksortSimplified(smaller)  + [pivot] + quicksortSimplified(larger)

In [ ]:
testdata = ["zebra", "bison", "dachs", "kuh", "zebra", "kuh"]
quicksortSimplified(testdata)

Als nächstes führen wir den 3-Weg Quicksort ein. Dabei wird das Array nicht nur in zwei Teile partitioniert, sondern in 3. Die Elemente die kleiner sind, die die grösser sind, sowie die, die gleich sind. 

In [ ]:
def partition3Ways(a, pivot):
    smaller = []
    equal = []
    larger = []
    for x in a:
        if x < pivot:
            smaller.append(x)
        elif x == pivot:
            equal.append(x)
        else:
            larger.append(x)
    return (smaller, equal, larger)

Die Strategie bleibt dieselbe: Wir sortieren einfach die einzelnen Teilarrays. Beachte aber, dass wir jetzt das Pivot Element nicht mehr separat hinzufügen, sondern dass dies im ```equal``` Teilarray enthalten ist. 

In [ ]:
def quicksort3WaySimplified(a):
    if len(a) <= 1:
        return a
    
    pivot = a[0]
    (smaller, equal, larger) = partition3Ways(a, pivot)  
    return quicksort3WaySimplified(smaller) \
            + equal   \
            + quicksort3WaySimplified(larger)

Dieser Algorithmus hat klare Vorteile, wenn wir viele redundante Elemente in der Liste haben, da diese dann im nächsten rekursiven Aufruf nicht mehr mitgeführt werden.

In [ ]:
testdata = ["zebra", "bison", "dachs", "kuh", "zebra", "kuh"]
quicksort3WaySimplified(testdata)

Diese sehr einfache Idee können wir uns zunutze machen, wenn wir Strings sortieren. Die idee ist, dass wir am Anfang nur das erste Zeichen anschauen (d.h. das Pivot Element ist das erste Zeichen vom ersten String), und anhand dessen sortieren. Dabei wird es viele Strings geben, die mit demselben Buchstaben anfangen. Diese sortieren wir dann rekursiv nach dem zweiten Buchstaben, etc. Bei allen anderen weiss man bereits nach dem Vergleich ob diese kleiner oder grösser dem Pivot Element sind. 

Um diese Strategie umzusetzen, fügen wir als erstes eine Hilfsfunktion ein, die uns für einen gegebenen string ```s``` und eine Position ```d``` das Zeichen an dieser Position zurückgibt. Falls ```d``` über das Ende vom String zeigt, wird das kleinste Zeichen im Alphabet zurückgegeben. Dank dieser Funktion können wir Strings unterschiedlicher Länge alle gleich behandeln.

In [ ]:
def charAt(s, d):
    if (d < len(s)):
        return s[d]
    else:
        return chr(0)

Die Partitionierungsfunktion nimmt jetzt zusätzlich als Argument die Position im String ```d```, die bestimmt, welches Zeichen verglichen wird

In [ ]:
def partition3WaysString(a, pivot, d):
    smaller = []
    equal = []
    larger = []
    
    charPivot = charAt(pivot, d)
    for x in a:
        charX = charAt(x, d)
        if charX < charPivot:
            smaller.append(x)
        elif charX == charPivot:
            equal.append(x)
        else:
            larger.append(x)
    return (smaller, equal, larger)

Im Gegensatz zum Standard 3-Weg Quicksort haben wir nun 3 rekursive Aufrufe. Nämlich müssen wir nun zusätzlich auch weitersortieren, wenn das Zeichen an Position ```d``` gleich wie das Zeichen im Pivotelement war. In diesem Fall müssen wir dann nach dem Zeichen an der Position ```d+1``` sortieren. 

In [ ]:

def quicksortStringSimplified(a, d):
    
    if len(a) <= 1 or d > len(a[0]):
        return a
    pivot = a[0]
        
    (smaller, equal, larger) = partition3WaysString(a, pivot, d)  
    
    return quicksortStringSimplified(smaller, d) \
            + quicksortStringSimplified(equal, d + 1) \
            + quicksortStringSimplified(larger, d)

Im folgenden testen wir unsere Implementation mit einem einfachen Testfall:

In [ ]:
xs = ["th","ti", "ha", "co", "fo", "al", "me", "to", "ri", "to", "th", "ch", "th", "is", "at", "ha"]
quicksortStringSimplified(xs, 0)

Die richtige (also nicht vereinfachte) Version von Quicksort für Strings funktioniert nach genau demselben Prinzip. Die Teilarrays werden hier einfach durch geschicktes setzen der hi und lo pointer definiert, wie in der frühreren Vorlesung besprochen. 

In [ ]:
def quicksort3WayInplace(a, lo, hi, d):  
    if hi <= lo:
        return
    
    (lt, gt) = (lo, hi) 
    pivot = charAt(a[lo], d);
    i = lo + 1;
    
    while i <= gt:
        t = charAt(a[i], d);
        if t < pivot:
            a[lt], a[i] = a[i], a[lt] 
            lt += 1
            i += 1 
        elif t > pivot: 
            a[i], a[gt] = a[gt], a[i]
            gt -= 1
        else:
            i += 1
            
    quicksort3WayInplace(a, lo, lt-1, d);
    if (pivot > chr(0)): quicksort3WayInplace(a, lt, gt, d+1);
    quicksort3WayInplace(a, gt+1, hi, d);    

Beim Aufruf müssen wir beachten, den Index für ```lo```, ```hi``` und ```d``` richtig zu setzen. (In der Praxis würden wir dem Benutzer natürlich eine zusätliche Funktion zur Verfügung stellen, die dies automatisch macht). 

In [ ]:
xs = ["th","ti", "ha", "co", "fo", "al", "me", "to", "ri", "to", "th", "ch", "th", "is", "at", "ha"]
quicksort3WayInplace(xs, 0, len(xs) - 1, 0)
print(xs)

## Experimente

Wir können nun die Algorithmen vergleichen. Uns interessieren insbesondere zwei Werte:

* Die Anzahl (Zeichen/String) Vergleiche
* Die Laufzeit

Unter anderem sind folgende Experimente interessant:

* Wie unterscheidet sich der Aufwand der Algorithmen auf zufälligen Strings
* Was passiert wenn die Strings lange gemeinsame Präfixe haben
* Wie beeinflusst die Grösse des Alphabets die Performance

In [ ]:
import random

def randomString(alphabet, n, commonPrefix=""):  
    s = commonPrefix
    for i in range(0, n):
        r = random.randint(0, alphabet.radix() -1)
        s += alphabet.toChar(r)
    return s
    

In [ ]:
aLongPrefix = randomString(basicAscii, 1)
randomStrings = [randomString(basicAscii, 300) for i in range(0, 1000)]


Nun können wir die Laufzeit beider Algorithmen vergleichen.

In [ ]:
from timeit import timeit
print("quicksort: ", timeit(lambda : quicksortStringSimplified(randomStrings, 0), number=100))
print("lsd: ", timeit(lambda : lsdSort(randomStrings), number=100))



*Übung:* Experimentieren Sie mit Strings zufälliger Länge und vergleichen Sie die Laufzeit. Varieren Sie auch die Länge des gemeinsamen Präfix